In [2]:
pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [32]:
import pgmpy.models
import pgmpy.inference

model = pgmpy.models.BayesianNetwork([('Chuva', 'Grama_Tania'),
                                      ('Irrigador', 'Grama_Tania'),
                                      ('Chuva', 'Grama_Joao')])

dpc_chuva = TabularCPD(variable='Chuva', variable_card=2,
                       values=[[0.8], [0.2]])

dpc_irrigador = TabularCPD(variable='Irrigador', variable_card=2,
                           values=[[0.9], [0.1]])

dpc_grama_tania = TabularCPD(variable='Grama_Tania', variable_card=2,
                             values=[[0.99, 0.8, 0.0, 1.0],
                                     [0.01, 0.2, 1.0, 0.0]],
                             evidence=['Chuva', 'Irrigador'],
                             evidence_card=[2, 2])

dpc_grama_joao = TabularCPD(variable='Grama_Joao', variable_card=2,
                            values=[[0.8, 0.2],
                                    [0.2, 0.8]],
                            evidence=['Chuva'],
                            evidence_card=[2])

model.add_cpds(dpc_chuva, dpc_irrigador, dpc_grama_tania, dpc_grama_joao)


In [33]:
model.check_model()


True

In [34]:

inferencia = pgmpy.inference.VariableElimination(model)

resultado = inferencia.query(variables=['Irrigador'],
                             evidence={'Grama_Joao': 1,'Grama_Tania': 1})

print("Probabilidade do irrigador estar ligado dado que a grama de joão e Tânia está molhada")
print(resultado)

Probabilidade do irrigador estar ligado dado que a grama de joão e Tânia está molhada
+--------------+------------------+
| Irrigador    |   phi(Irrigador) |
+==============+==================+
| Irrigador(0) |           0.9785 |
+--------------+------------------+
| Irrigador(1) |           0.0215 |
+--------------+------------------+


In [35]:
resultado = inferencia.query(variables=['Irrigador'],
                             evidence={'Grama_Joao': 0,'Grama_Tania': 1})

print("Probabilidade do irrigador estar ligado dado que a grama do João está seca e da Tânia está molhada")
print(resultado)

Probabilidade do irrigador estar ligado dado que a grama do João está seca e da Tânia está molhada
+--------------+------------------+
| Irrigador    |   phi(Irrigador) |
+==============+==================+
| Irrigador(0) |           0.7654 |
+--------------+------------------+
| Irrigador(1) |           0.2346 |
+--------------+------------------+


In [36]:
resultado = inferencia.query(variables=['Chuva'],
                             evidence={'Grama_Joao': 1,'Grama_Tania': 1})

print("Probabilidade de estar chovendo dado que a grama do João e da Tânia está molhada")
print(resultado)

Probabilidade de estar chovendo dado que a grama do João e da Tânia está molhada
+----------+--------------+
| Chuva    |   phi(Chuva) |
+==========+==============+
| Chuva(0) |       0.0312 |
+----------+--------------+
| Chuva(1) |       0.9688 |
+----------+--------------+


In [37]:
resultado = inferencia.query(variables=['Chuva'],
                             evidence={'Grama_Joao': 0,'Grama_Tania': 1})

print("Probabilidade de estar chovendo dado que a grama do João está seca e da Tânia está molhada")
print(resultado)

Probabilidade de estar chovendo dado que a grama do João está seca e da Tânia está molhada
+----------+--------------+
| Chuva    |   phi(Chuva) |
+==========+==============+
| Chuva(0) |       0.3402 |
+----------+--------------+
| Chuva(1) |       0.6598 |
+----------+--------------+
